In [1]:
import norfetools as nt
import osmnx as ox
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import shapely.geometry as sg

import numpy as np
import os

from Global import cities, api_key
from Global import keywords_list, types_list
from shapely.geometry import MultiPolygon, Polygon

In [2]:
import requests

def find_nearby_stores(latitude, longitude, api_key):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': f'{latitude},{longitude}',
        'radius': '10000',  # 搜索半径（以米为单位）
        'type': 'convenience_store',  # 搜索便利店
        'key': api_key
    }
    
    response = requests.get(base_url, params=params)
    store_locations = []  # 用来存储每个便利店的经纬度位置

    if response.status_code == 200:
        results = response.json()
        stores = results.get('results', [])
        if stores:
            for store in stores:
                # 获取便利店的位置信息
                location = store.get('geometry', {}).get('location')
                if location:
                    lat = location.get('lat')
                    lng = location.get('lng')
                    store_locations.append((lat, lng))
        return store_locations
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None


In [3]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def find_places(latitude, longitude, api_key, place_type=None, keyword=None):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 创建 Session 对象，并设置重试策略
    session = requests.Session()
    retries = Retry(total=5,  # 总共重试的次数
                    backoff_factor=1,  # 等待时间的退避因子
                    status_forcelist=[500, 502, 503, 504])  # 指定哪些状态码需要重试
    session.mount('https://', HTTPAdapter(max_retries=retries))
    
    params = {
        'location': f'{latitude},{longitude}',
        'radius': '10000',  # 搜索半径（以米为单位）
        'key': api_key
    }
    
    if place_type:
        params['type'] = place_type
    if keyword:
        params['keyword'] = keyword
    
    try:
        response = session.get(base_url, params=params)
        location_list = []  # 用于存储地点的经纬度信息

        if response.status_code == 200:
            results = response.json()
            places = results.get('results', [])
            for place in places:
                location = place.get('geometry', {}).get('location')
                if location:
                    lat = location.get('lat')
                    lng = location.get('lng')
                    location_list.append((lat, lng))
            return location_list
        else:
            print(f"Failed to retrieve data: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [4]:
import time
from tqdm import tqdm

for city_name in cities:

    data = nt.Load_data("./Data/@boundary/%s.pkl"%city_name)

    for loc_type in types_list:
        file_name = f"{city_name}/Type_{loc_type}.pkl"
        all_locations = []
        for loc in tqdm(data[1]):
            lat, lon = loc
            locations = find_places(lat, lon, api_key, place_type=loc_type)
            all_locations.extend(locations)
            time.sleep(0.2)
        all_locations = list(set(all_locations))
            
        nt.Save_data(all_locations, "Data/%s"%file_name)
        print(f"{file_name} saved {len(all_locations)} locations.")

    for loc_key in keywords_list:
        file_name = f"{city_name}/Keywords_{loc_key}.pkl"
        all_locations = []
        for loc in tqdm(data[1]):
            lat, lon = loc
            locations = find_places(lat, lon, api_key, keyword=loc_key)
            all_locations.extend(locations)
            time.sleep(1)
        all_locations = list(set(all_locations))
            
        nt.Save_data(all_locations, "Data/%s"%file_name)
        print(f"{file_name} saved {len(all_locations)} locations.")

  0%|          | 0/232 [00:00<?, ?it/s]

100%|██████████| 232/232 [05:01<00:00,  1.30s/it]


Amsterdam/Type_convenience_store.pkl saved 77 locations.


100%|██████████| 232/232 [04:49<00:00,  1.25s/it]


Amsterdam/Type_lodging.pkl saved 105 locations.


...

Singapore/Type_post_office.pkl saved 208 locations.


100%|██████████| 209/209 [05:50<00:00,  1.68s/it]


Singapore/Type_police.pkl saved 147 locations.


 11%|█         | 22/209 [05:20<45:21, 14.56s/it]


KeyboardInterrupt: 